In [77]:
from bs4 import BeautifulSoup as soup
import requests
import re
import pandas as pd

In [78]:
movie_info = {}
movie_info['movie_title'] = []
movie_info['genre'] = []
movie_info['director'] = []
movie_info['cast'] = []

counter = 0

In [79]:
def get_movies(url):
    text = requests.get(url)
    text = text.text
    file = soup(text, 'html.parser')

    block = file.find('div', class_ = 'lister-list') #finding the lister-list class
    movies = block.find_all('div', class_ = 'lister-item-content') #list of all the movies content

    next = file.find('div', class_ = 'desc') #refrence for next page
    next = "https://www.imdb.com/" + next.find('a',class_ = 'lister-page-next next-page')['href'] #next page link

    return movies, next

In [80]:
import time
import os

def loop_moives(movies, next_page):
    for movie in movies:

        ##==================================================================
        try:
            movie_name = movie.select('a[href^="/title/"]')[0].string #movie title
        except:
            movie_name = '-1'

        ##==================================================================
        try:
            genre = movie.find('p', class_ = 'text-muted')
            genre = genre.find('span', class_ = 'genre').string.strip() #genre list
        except:
            genre = '-1'

        #==================================================================
        try:
            directors = movie.find('p', class_ = '')
            directors = str(directors).split('Stars')[0]
            directors = soup(directors, 'html.parser')
            directors = directors.select('a[href^="/name/"]') 

            all_directors = ""
            for director in directors:
                all_directors += director.string
                all_directors += ', '
            
            all_directors = all_directors[:-2] #director name
        except:
            all_directors = '-1'

        ##==================================================================
        try:
            casts = movie.find('p', class_ = '')
            casts = str(casts).split('Stars')[1]
            casts = soup(casts, 'html.parser')
            casts = casts.select('a[href^="/name/"]') 
            
            
            all_cast = ""
            for cast in casts:
                all_cast += cast.string
                all_cast += ', '
            
            all_cast = all_cast[:-2] #all cast names
        except:
            all_cast = '-1'

        ##==================================================================

        movie_info['movie_title'].append(movie_name)
        movie_info['genre'].append(genre)
        movie_info['director'].append(all_directors)
        movie_info['cast'].append(all_cast)


    if counter%10 == 0:
        print(f'Done with Page {counter}')
    return next_page

In [81]:
first_link = "https://www.imdb.com/search/title/?year=2021&title_type=feature&"

while first_link:
    movies_list, next_page = get_movies(first_link)
    first_link = loop_moives(movies_list, next_page)

    counter +=1

Done with Page 0
Done with Page 10
Done with Page 20
Done with Page 30
Done with Page 40
Done with Page 50
Done with Page 60
Done with Page 70
Done with Page 80


In [ ]:
movies = pd.DataFrame.from_dict(movie_info)
print(movies.shape)

movies.to_csv('2021_movies_imdb.csv', index=False)

,movie_title,genre,director,cast
0,Don't Look Up,"Comedy, Drama, Sci-Fi",Adam McKay,"Leonardo DiCaprio, Jennifer Lawrence, Meryl St..."
1,Cruella,"Adventure, Comedy, Crime",Craig Gillespie,"Emma Stone, Emma Thompson, Joel Fry, Paul Walt..."
2,Mystère,Drama,Denis Imbert,"Vincent Elbaz, Shanna Keil, Marie Gillain, Eri..."
3,De uskyldige,"Drama, Horror, Mystery",Eskil Vogt,"Rakel Lenora Fløttum, Alva Brynsmo Ramstad, Sa..."
4,Sooryavanshi,"Action, Crime, Thriller",Rohit Shetty,"Akshay Kumar, Katrina Kaif, Ajay Devgn, Ranvee..."
5,Pleasure,Drama,Ninja Thyberg,"Sofia Kappel, Revika Anne Reustle, Evelyn Clai..."
6,Bliss,"Drama, Romance",Mike Cahill,"Owen Wilson, Salma Hayek, Nesta Cooper, Jorge ..."
7,Son,"Horror, Thriller",Ivan Kavanagh,"Andi Matichak, Emile Hirsch, Luke David Blumm,..."
